In [30]:
import pandas as pd

# 讀取 Excel 檔案
excel_file = '' #Excel File path
relative_path = '' #txt file path
df = pd.read_excel(excel_file)
# 儲存為 .txt 檔案，使用 , 或 | 進行區隔
with open(relative_path, 'w', encoding='utf-8') as f:
    for index, row in df.iterrows():
        f.write('|'.join(row.astype(str)) + '\n')
        f.write('-' * 50 + '\n')  # 用 - 進行區隔

print(f'資料已儲存到 {relative_path}')

資料已儲存到 D:\Project\2024_POC\MOL_GPT_0821\問答集-20240814-測試.txt


In [31]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [33]:
relative_path = '' #File folder path
# relative_path = '' #File path

text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(relative_path, glob="**/*.txt", show_progress=True,loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
# doc_sources = [doc.metadata['source']  for doc in docs]
# loader = TextLoader(relative_path,autodetect_encoding = True)
docs = loader.load()
# docs = []
# for loader in loaders:
#     docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = 1024,
    chunk_overlap  = 50
    )
spl_text = text_splitter.split_documents(docs)

100%|██████████| 1/1 [00:00<00:00, 200.55it/s]


In [34]:
# Define the path to the pre-trained model you want to use
#### Most Download model
modelPath = 'sentence-transformers/all-MiniLM-L6-v2'
#### BU3 text2vec
# modelPath = "shibing624/text2vec-base-chinese"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)
# db = FAISS.from_documents(spl_text, embeddings)

d:\Project\Virtual_Enviroment\conda_venv\HuggingFace\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [35]:
from langchain.chat_models import AzureChatOpenAI
# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = AzureChatOpenAI(
    openai_api_base = "",
    openai_api_version="",
    azure_deployment="",
    openai_api_key="",
    openai_api_type="",
    temperature = 0.5,
    max_tokens = 1024,
    streaming=True
    )

d:\Project\Virtual_Enviroment\conda_venv\HuggingFace\lib\site-packages\langchain_community\chat_models\azure_openai.py:168: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://gpt-east-usa.openai.azure.com/ to https://gpt-east-usa.openai.azure.com/openai.
  warnings.warn(
d:\Project\Virtual_Enviroment\conda_venv\HuggingFace\lib\site-packages\langchain_community\chat_models\azure_openai.py:175: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
d:\Project\Virtual_Enviroment\conda_venv\HuggingFace\lib\site-packages\langchain_community\chat_models\azure_openai.py:183: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specifie

# Ensemble Retriever
The EnsembleRetriever takes a list of retrievers as input and ensemble the results of their get_relevant_documents() methods and rerank the results based on the Reciprocal Rank Fusion algorithm.

By leveraging the strengths of different algorithms, the EnsembleRetriever can achieve better performance than any single algorithm.

The most common pattern is to combine a sparse retriever (like BM25) with a dense retriever (like embedding similarity), because their strengths are complementary. It is also known as "hybrid search". The sparse retriever is good at finding relevant documents based on keywords, while the dense retriever is good at finding relevant documents based on semantic similarity.

In [36]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
# doc_list_1 = [
#     "I like apples",
#     "I like oranges",
#     "Apples and oranges are fruits",
# ]

# # initialize the bm25 retriever and faiss retriever
# bm25_retriever = BM25Retriever.from_texts(
#     doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
# )
# bm25_retriever.k = 2

# doc_list_2 = [
#     "You like apples",
#     "You like oranges",
# ]

# embedding = embeddings
# faiss_vectorstore = FAISS.from_texts(
#     doc_list_2, embedding, metadatas=[{"source": 2}] * len(doc_list_2)
# )
# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_documents(
    spl_text
)
bm25_retriever.k = 3

embedding = embeddings
faiss_vectorstore = FAISS.from_documents(
    spl_text, embedding
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 3})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

# Cross Encoder Reranker
This notebook shows how to implement reranker in a retriever with your own cross encoder from Hugging Face cross encoder models or Hugging Face models that implements cross encoder function (example: BAAI/bge-reranker-base). SagemakerEndpointCrossEncoder enables you to use these HuggingFace models loaded on Sagemaker.

This builds on top of ideas in the ContextualCompressionRetriever. Overall structure of this document came from Cohere Reranker documentation.

For more about why cross encoder can be used as reranking mechanism in conjunction with embeddings for better retrieval, refer to Hugging Face Cross-Encoders documentation.

# Doing reranking with CrossEncoderReranker
Now let's wrap our base retriever with a ContextualCompressionRetriever. CrossEncoderReranker uses HuggingFaceCrossEncoder to rerank the returned results.

In [37]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

# 查看查詢出來的文件內容
# compressed_docs = compression_retriever.invoke("<QUESTION>")


In [40]:
# docs = ensemble_retriever.invoke("RAG")
question = "這位38歲的男性目前處於失業狀態，已經失業了3到6個月。他在失業後決定自主創業，計劃開設一間小型咖啡廳，目標是為社區居民提供便捷的咖啡服務並開展外送業務。有哪些推薦的申報資訊可以告訴我"
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=compression_retriever, return_source_documents=False)
# print(chain.run(question))
# 執行查詢並使用流式輸出
def run_with_streaming(chain, question):
    response = chain.run(question)
    for chunk in response:
        print(chunk, end="", flush=True)

run_with_streaming(chain, question)

如果這位38歲的男性計劃開設一家小型咖啡廳並開展外送業務，他需要了解和準備一些重要的申報資訊。以下是一些關鍵的步驟和建議：

1. **商業登記**：
   - **公司註冊**：確保將咖啡廳註冊為合法的商業實體。可以選擇註冊為獨資經營、合夥企業或有限公司。
   - **營業登記**：向當地政府申請營業執照，確保合法經營。

2. **稅務申報**：
   - **稅務登記**：向稅務部門登記，獲取稅務識別號碼（TIN）。
   - **營業稅**：了解當地的營業稅規定，並確保按時申報和繳納。
   - **所得稅**：了解個人和企業所得稅的申報要求。

3. **衛生和安全許可**：
   - **食品衛生許可**：申請食品衛生許可證，確保咖啡廳符合食品安全標準。
   - **店鋪設計和設備檢查**：確保店鋪設計和設備符合當地的衛生和安全規範。

4. **勞動法規和員工管理**：
   - **勞工保險和福利**：為員工提供必要的勞工保險和福利，並遵守勞動法規。
   - **員工訓練**：對員工進行必要的培訓，特別是食品安全和客戶服務方面。

5. **營運許可和其他相關證照**：
   - **營運許可**：根據當地法律，申請其他可能需要的營運許可，例如外送業務的相關許可。
   - **廣告和標識許可**：如果需要設置戶外廣告或標識，可能需要申請相關的許可。

6. **財務和會計管理**：
   - **會計記錄**：保持詳細的財務記錄，確保所有收入和支出都被記錄和報告。
   - **財務報表**：定期編制財務報表，了解咖啡廳的經營狀況。

7. **市場行銷和推廣**：
   - **市場調查**：了解目標市場和競爭對手，制定合適的市場行銷策略。
   - **社交媒體和在線推廣**：利用社交媒體和在線平台進行推廣，吸引更多客戶。

這些步驟和資訊可以幫助他順利開設和運營咖啡廳，並確保符合所有相關法規和要求。建議他諮詢專業的商業顧問或律師，以獲得更具體和詳細的指導。